# Imports

In [76]:
import fastparquet as fpq
import math
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import polars as pl
import sys

from pathlib import Path

In [60]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# Constants

In [61]:
TRAINING_SIZE = .7
VALIDATION_SIZE = .15
MBTA_PATH = Path("MBTA")

# data retrieved from https://mbta-massdot.opendata.arcgis.com/datasets/MassDOT::mbta-bus-arrival-departure-times-2022/about
MBTA_PATH.mkdir(parents=True, exist_ok=True)
MBTA_CSV = MBTA_PATH / "MBTA-Bus-Arrival-Departure-Times_2022-05.csv"
MBTA_PARQUET = MBTA_PATH / "MBTA-Bus-Arrival-Departure-Times_2022-05.parquet"

# Building a model generator

In [62]:
# Build sample data
data_set = list(range(1,100))
data_size = len(data_set)
# Training data set
# - Use this set for model training
# - 70–80% of your data is the standard
trng_limit = math.ceil(data_size*TRAINING_SIZE)
training_data = data_set[:trng_limit]

# Validation/development data set
# - Use this set for model hyperparameter tuning and experimentation evaluation
# - 10–15% of your data is the standard
val_size = math.ceil(data_size*VALIDATION_SIZE)
val_limit = trng_limit+val_size
validation_data = data_set[trng_limit:val_limit]

# Test data set
# - Use this set for model testing and comparison
# - 10–15% of your data is the standard.
# - Training and test data should be separated.
# - Test data shouldnt be used for model improvement
test_data = data_set[val_limit:]

print(data_set)
print(training_data)
print(validation_data)
print(test_data)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
[71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]
[86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [63]:
class data_model():
    data = []
    training_data = []
    validation_data = []
    test_data = []
    
    _TRAINING_SIZE = 0
    _VALIDATION_SIZE = 0    
    
    def __init__(self, training_size=.70, validation_size=.15):
        self._TRAINING_SIZE = training_size
        self._VALIDATION_SIZE = validation_size
    
    
    def build_models(self, data):
        self.data = data
        data_size = len(self.data)
        
        trng_limit = math.ceil(data_size*self._TRAINING_SIZE)
        self.training_data = data[:trng_limit]
        
        val_size = math.ceil(data_size*self._VALIDATION_SIZE)
        val_limit = trng_limit+val_size        
        self.validation_data = data[trng_limit:val_limit]
        
        self.test_data = data[val_limit:]           

In [64]:
datamodel = data_model(.8,.1)
datamodel.build_models(data_set)

print(datamodel.training_data)
print(datamodel.validation_data)
print(datamodel.test_data)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
[81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
[91, 92, 93, 94, 95, 96, 97, 98, 99]


# Working with Polars
### Ref: https://pythonspeed.com/articles/polars-memory-pandas/

## Create the parquet file

In [65]:
df = pd.read_csv(MBTA_CSV,
                dtype={"route_id":"string",},
                parse_dates=["service_date","scheduled","actual"],)

df.dtypes

service_date         datetime64[ns]
route_id                     string
direction_id                 object
half_trip_id                float64
stop_id                       int64
time_point_id                object
time_point_order              int64
point_type                   object
standard_type                object
scheduled            datetime64[ns]
actual               datetime64[ns]
scheduled_headway           float64
headway                     float64
dtype: object

In [66]:
for categorical_col in ["route_id","direction_id","point_type","standard_type"]:
    df[categorical_col] = df[categorical_col].astype("category")

In [67]:
df.to_parquet(str(MBTA_CSV).replace(".csv",".parquet"))

# Use Naive Pandas

In [68]:
def find_worst_headways_pandas():
    # Load the data
    data = pd.read_parquet(MBTA_PARQUET)
    # Filter down to headway points only
    data[data["standard_type"] == "headway"]
    # Calc ratio of actual headway to expected headway
    data["headway_ratio"] = data.headway / data.scheduled_headway
    # Group by route and direction (inbound/outbound)
    by_route = data.groupby(["route_id","direction_id"])
    # Find median headway ratio for each route
    median_headway = by_route[["headway_ratio"]].median()
    # Return the worst 5 routes:
    return median_headway.nlargest(5, columns=["headway_ratio"])

In [69]:
%%timeit
%%memit
find_worst_headways_pandas()

peak memory: 1558.98 MiB, increment: 653.30 MiB
peak memory: 1519.84 MiB, increment: 197.69 MiB
peak memory: 1423.19 MiB, increment: 380.68 MiB
peak memory: 1552.89 MiB, increment: 422.68 MiB
peak memory: 1538.85 MiB, increment: 331.45 MiB
peak memory: 1537.22 MiB, increment: 586.12 MiB
peak memory: 1456.21 MiB, increment: 100.11 MiB
peak memory: 1518.45 MiB, increment: 532.35 MiB
874 ms ± 80.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## A More Optimized Pandas Implementation

In [70]:
def find_worst_headways_pandas_opt():
    # Load the data in chunks
    chunks = []
    parquet_file = pq.ParquetFile(MBTA_PARQUET)
    
    for batch in parquet_file.iter_batches():
        chunk = batch.to_pandas()
        del batch
        # Calculate headway ratio:
        chunk["headway_ratio"] = chunk.headway / chunk.scheduled_headway
        # Store the columns we care about for this chunk
        chunks.append(chunk[["route_id","direction_id","headway_ratio"]])
    
    del parquet_file
    
    # Concatenate into one big DataFrame
    # Not ideal, involves two copies in memory at once
    data = pd.concat(chunks)
    del chunks
    
    # Group by route and direction (inbound/outbound)
    by_route = data.groupby(["route_id","direction_id"])
    # Find median day's headway ratio for each route
    median_headway = by_route[["headway_ratio"]].median()
    # Return the worst 5 routes
    return median_headway.nlargest(5, columns=["headway_ratio"])

In [71]:
%%timeit
%%memit
find_worst_headways_pandas_opt()

peak memory: 1321.98 MiB, increment: 0.00 MiB
peak memory: 915.65 MiB, increment: 49.63 MiB
peak memory: 1026.07 MiB, increment: 151.88 MiB
peak memory: 1023.53 MiB, increment: 128.78 MiB
peak memory: 1011.85 MiB, increment: 141.62 MiB
peak memory: 1027.65 MiB, increment: 164.15 MiB
peak memory: 1019.18 MiB, increment: 130.56 MiB
peak memory: 1024.36 MiB, increment: 150.79 MiB
964 ms ± 8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
def find_worst_headways_pandas_fast():
    # Load the data
    data = pd.read_parquet(MBTA_PARQUET, engine="fastparquet")
    # Filter down to headway points only
    data[data["standard_type"] == "headway"]
    # Calc ratio of actual headway to expected headway
    data["headway_ratio"] = data.headway / data.scheduled_headway
    # Group by route and direction (inbound/outbound)
    by_route = data.groupby(["route_id","direction_id"])
    # Find median headway ratio for each route
    median_headway = by_route[["headway_ratio"]].median()
    # Return the worst 5 routes:
    return median_headway.nlargest(5, columns=["headway_ratio"])

In [73]:
def find_worst_headways_pandas_opt_fast():
    # Load the data in chunks
    chunks = []
    parquet_file = fpq.ParquetFile(MBTA_PARQUET)
    
    for chunk in parquet_file.iter_row_groups():    
        # Calculate headway ratio:
        chunk["headway_ratio"] = chunk.headway / chunk.scheduled_headway
        # Store the columns we care about for this chunk
        chunks.append(chunk[["route_id","direction_id","headway_ratio"]])
    
    del parquet_file
    
    # Concatenate into one big DataFrame
    # Not ideal, involves two copies in memory at once
    data = pd.concat(chunks)
    del chunks
    
    # Group by route and direction (inbound/outbound)
    by_route = data.groupby(["route_id","direction_id"])
    # Find median day's headway ratio for each route
    median_headway = by_route[["headway_ratio"]].median()
    # Return the worst 5 routes
    return median_headway.nlargest(5, columns=["headway_ratio"])

In [74]:
%%timeit
%%memit
find_worst_headways_pandas_fast()

peak memory: 1109.43 MiB, increment: 240.99 MiB
peak memory: 1080.45 MiB, increment: 271.53 MiB
peak memory: 1080.45 MiB, increment: 271.76 MiB
peak memory: 1080.20 MiB, increment: 271.51 MiB
peak memory: 1080.45 MiB, increment: 271.77 MiB
peak memory: 1080.46 MiB, increment: 271.77 MiB
peak memory: 1071.96 MiB, increment: 263.28 MiB
peak memory: 1075.96 MiB, increment: 267.28 MiB
1.54 s ± 8.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
%%timeit
%%memit
find_worst_headways_pandas_opt_fast()

peak memory: 1008.09 MiB, increment: 199.40 MiB
peak memory: 1050.04 MiB, increment: 241.35 MiB
peak memory: 1042.04 MiB, increment: 233.36 MiB
peak memory: 1010.26 MiB, increment: 201.57 MiB
peak memory: 1010.26 MiB, increment: 201.57 MiB
peak memory: 1020.00 MiB, increment: 211.31 MiB
peak memory: 1072.07 MiB, increment: 263.39 MiB
peak memory: 1062.04 MiB, increment: 253.35 MiB
1.55 s ± 9.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Polars Implementation

In [79]:
def headways_sorted_worst_first():
    # Load the data lazily
    data = pl.scan_parquet(MBTA_PARQUET)
    # Filter down to headway points only and then select
    # the data we need
    data = data.filter(pl.col("standard_type") == "Headway"
                      ).select([
                            pl.col("route_id"),
                            pl.col("direction_id"),
                            pl.col("headway") / pl.col("scheduled_headway"),
                        ])
    
    # Group by route and direction (inbound/outbound)
    by_route = data.groupby(["route_id","direction_id"])
    # Find the median headway ratio for each route
    median_headway = by_route.agg(pl.col("headway").median())
    # Theres no nlargest() method, so instead just sort in descending order
    return median_headway.sort("headway", reverse=True)

In [82]:
%%timeit
%%memit
# Create the query
query = headways_sorted_worst_first()
# Actually run the query
result = query.collect()
# Print the 5 worst headways:
print(result[:5, :])

shape: (5, 3)
┌──────────┬──────────────┬──────────┐
│ route_id ┆ direction_id ┆ headway  │
│ ---      ┆ ---          ┆ ---      │
│ cat      ┆ cat          ┆ f64      │
╞══════════╪══════════════╪══════════╡
│ 108      ┆ Outbound     ┆ 2.9      │
│ 88       ┆ Outbound     ┆ 1.68     │
│ 83       ┆ Outbound     ┆ 1.565    │
│ 134      ┆ Outbound     ┆ 1.431111 │
│ 134      ┆ Inbound      ┆ 1.346667 │
└──────────┴──────────────┴──────────┘
peak memory: 943.31 MiB, increment: 12.02 MiB
shape: (5, 3)
┌──────────┬──────────────┬──────────┐
│ route_id ┆ direction_id ┆ headway  │
│ ---      ┆ ---          ┆ ---      │
│ cat      ┆ cat          ┆ f64      │
╞══════════╪══════════════╪══════════╡
│ 108      ┆ Outbound     ┆ 2.9      │
│ 88       ┆ Outbound     ┆ 1.68     │
│ 83       ┆ Outbound     ┆ 1.565    │
│ 134      ┆ Outbound     ┆ 1.431111 │
│ 134      ┆ Inbound      ┆ 1.346667 │
└──────────┴──────────────┴──────────┘
peak memory: 957.12 MiB, increment: 13.80 MiB
shape: (5, 3)
┌───────